In [18]:
from games.utils.query_reader import QueryReader
import pandas as pd
import psycopg2
import time
import os

query_dir = os.path.join(os.path.abspath(''), 'queries')
qr = QueryReader(query_dir)
BASE_DATE = "'2021-01-01'"

def create_connection(): 
    return psycopg2.connect(
        dbname='main',
        host='main-us-e2.cmbsiiqeauby.us-east-2.rds.amazonaws.com',
        port=5432,
        user='doadmin',
        password='i39kew8n7jcat7l9'
    )

def run_query(connection, query):
    cursor = connection.cursor()
    try:
        start_time = time.time()
        cursor.execute(query)
        results = cursor.fetchall()
        end_time = time.time()

        if len(results[0]) == 1:
            results = [x[0] for x in results]
        else:
            results = list(list(x) for x in results)
        print(f'Time elapsed: {end_time - start_time}')
        print(f'Length of Results: {len(results)}')
        print(f'Sample Result: {results[0]}')
        return results
    except:
        print('rolling back')
        connection.rollback()
    finally:
        cursor.close()


def read_into_df(connection, query) -> pd.DataFrame:
    start_time = time.time()
    df = pd.read_sql(connection, query)
    end_time = time.time()
    print(f'Time elapsed: {end_time - start_time}')
    return df


conn = create_connection()

In [23]:
raw_map_ids_query = qr.read_query('map_id.sql')
map_ids_query = raw_map_ids_query.format(date=BASE_DATE) 

map_ids = run_query(conn, map_ids_query)
str_map_ids = ','.join([str(x) for x in map_ids])



Time elapsed: 0.8888423442840576
Length of Results: 16197
Sample Result: 2


In [24]:
raw_ck1 = qr.read_query('csgo_kills copy.sql')
ck1 = raw_ck1.format(map_ids=str_map_ids)
results = run_query(conn, ck1)

In [20]:
raw_ck2 = qr.read_query('csgo_kills.sql')
ck2 = raw_ck2.format(date=BASE_DATE)
results = run_query(conn, ck2)

Time elapsed: 17.661826848983765
Length of Results: 74266
Sample Result: [0, 76561197960710573, 16, Decimal('16.0000'), Decimal('1.0000'), Decimal('1.0000'), Decimal('9.0000'), Decimal('9.0000'), 6, Decimal('6.0000'), 0, Decimal('0.0000'), 0, Decimal('0.0000'), 0, Decimal('0.0000'), 0, Decimal('0.0000'), 1, Decimal('1.0000'), 9, Decimal('9.0000')]
